# A1_4 – Regularisation

In this notebook we compare different regularisation techniques applied to our neturalnet_torch model.

More specifically, we compare:

- L1/L2 Regularisation
- Dropout Regularisation

For each:

- We experiment with different parameters
- We present the results of the evaluation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import joblib

base = os.path.dirname(os.getcwd())  
sys.path.append(os.path.join(base, "models"))
sys.path.append(os.path.join(base, "utils"))

from sklearn.metrics import mean_squared_error, mean_absolute_error

from utils import predict_batch, mape, evaluate_regression
from NeuralNet import NeuralNet                      # manual BP implementation
from mlr_sklearn import MultipleLinearRegressionSK   # simple MLR wrapper
from neuralnet_torch import NeuralNetTorch           # PyTorch implementation

In [2]:
# Load preprocessed data from ./data
X_trainval_np = np.load("../data/X_trainval_np.npy")
X_test_np     = np.load("../data/X_test_np.npy")

y_trainval = np.load("../data/y_trainval.npy")
y_test     = np.load("../data/y_test.npy")

y_trainval_scaled = np.load("../data/y_trainval_scaled.npy")
y_test_scaled     = np.load("../data/y_test_scaled.npy")

x_scaler = joblib.load("../data/x_scaler.joblib")
y_scaler = joblib.load("../data/y_scaler.joblib")

n_features = X_trainval_np.shape[1]
print("Loaded preprocessed data from ../data")
print("X_trainval_np:", X_trainval_np.shape)
print("X_test_np    :", X_test_np.shape)
print("n_features   :", n_features)

Loaded preprocessed data from ../data
X_trainval_np: (1200, 61)
X_test_np    : (300, 61)
n_features   : 61


In [3]:
# Selected configuration for manual BP, we copy here the desired Hyperpaarameters from the notebook 2

hidden_layers_bp = [40, 15]
epochs_bp = 600
lr_bp = 0.005
momentum_bp = 0.9
activation_bp = "tanh"

print("Manual BP selected configuration:")
print("Hidden layers :", hidden_layers_bp)
print("Epochs        :", epochs_bp)
print("Learning rate :", lr_bp)
print("Momentum      :", momentum_bp)
print("Activation    :", activation_bp)


Manual BP selected configuration:
Hidden layers : [40, 15]
Epochs        : 600
Learning rate : 0.005
Momentum      : 0.9
Activation    : tanh


In [10]:
# PyTorch Neural Network 

hidden_layers_torch = hidden_layers_bp
layers_torch = [n_features] + hidden_layers_torch + [1]

net_torch = NeuralNetTorch(
    n=layers_torch,
    fact=activation_bp,   # same activation
    eta=lr_bp,            # same learning rate
    alpha=momentum_bp,    # same momentum
    epochs=epochs_bp,     # same number of epochs
    val_split=0.2         # same validation split
)

regularisation = ['None', 'L1', 'L2']
results = []   

for reg in regularisation:
    print("\nTesting regularisation:", reg)
    
    # Train with scaled data
    net_torch.fit(X_trainval_np, y_trainval_scaled, reg)

    # Loss history for later plots
    train_err_torch, val_err_torch = net_torch.loss_epochs()
    
    # Predictions in scaled space
    y_trainval_pred_torch_scaled = net_torch.predict(X_trainval_np).reshape(-1, 1)
    y_test_pred_torch_scaled     = net_torch.predict(X_test_np).reshape(-1, 1)
    
    # Back to original target scale (cnt_log)
    y_trainval_pred_torch = y_scaler.inverse_transform(y_trainval_pred_torch_scaled).ravel()
    y_test_pred_torch     = y_scaler.inverse_transform(y_test_pred_torch_scaled).ravel()
    
    # Metrics (in original cnt_log scale)
    metrics_trainval = evaluate_regression(y_trainval, y_trainval_pred_torch)
    metrics_test     = evaluate_regression(y_test,     y_test_pred_torch)

    print("=== PyTorch Neural Network (same config) ===")
    print("TRAIN+VAL:", metrics_torch_trainval)
    print("TEST     :", metrics_torch_test)

    # Store results
    # Add TRAIN+VAL row
    results.append({
        "Regularisation": reg,
        "Split": "Train+Val",
        "MSE":  metrics_trainval["MSE"],
        "MAE":  metrics_trainval["MAE"],
        "MAPE": metrics_trainval["MAPE"],
    })

    # Add TEST row
    results.append({
        "Regularisation": reg,
        "Split": "Test",
        "MSE":  metrics_test["MSE"],
        "MAE":  metrics_test["MAE"],
        "MAPE": metrics_test["MAPE"],
    })


NeuralNetTorch (PyTorch) initialized
 - Layers: [61, 40, 15, 1]
 - Activation: tanh
 - Learning rate: 0.005 | Momentum: 0.9
 - Epochs: 600 | Val split: 0.2

Testing regularisation: None
Epoch 0: Train MSE=1.050383 | Val MSE=1.119804
Epoch 100: Train MSE=0.090887 | Val MSE=0.111286
Epoch 200: Train MSE=0.076372 | Val MSE=0.099934
Epoch 300: Train MSE=0.066288 | Val MSE=0.092341
Epoch 400: Train MSE=0.057634 | Val MSE=0.086009
Epoch 500: Train MSE=0.049924 | Val MSE=0.080445
=== PyTorch Neural Network (same config) ===
TRAIN+VAL: {'MSE': 0.07876244510237074, 'MAE': 0.1944299341935785, 'MAPE': 6.85705512755216}
TEST     : {'MSE': 0.12720147620255262, 'MAE': 0.22979684749645599, 'MAPE': 9.380183141727464}

Testing regularisation: L1
Epoch 0: Train MSE=0.050986 | Val MSE=0.052019
Epoch 100: Train MSE=0.043431 | Val MSE=0.051435
Epoch 200: Train MSE=0.037204 | Val MSE=0.047063
Epoch 300: Train MSE=0.031407 | Val MSE=0.043041
Epoch 400: Train MSE=0.026244 | Val MSE=0.039745
Epoch 500: Train M

In [12]:

# Comparison tables: TRAIN+VAL and TEST metrics
df_results = pd.DataFrame(results)

print("=== Training metrics w/ Regularisation ===")
display(df_results)




=== Training metrics w/ Regularisation ===


,Regularisation,Split,MSE,MAE,MAPE
0,None,Train+Val,0.097742,0.197781,7.445139
1,None,Test,0.138763,0.228323,9.770781
2,L1,Train+Val,0.040916,0.128934,4.827589
3,L1,Test,0.082052,0.169946,7.757051
4,L2,Train+Val,0.026633,0.101997,3.738414
5,L2,Test,0.073022,0.152818,7.291093


In [13]:

# Loss curves for manual BP and PyTorch

plt.figure()
plt.plot(train_err_manual, label="Manual BP - Train MSE")
if any(e is not None for e in val_err_manual):
    plt.plot([e for e in val_err_manual if e is not None], label="Manual BP - Val MSE")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.title("Manual NeuralNet - Loss per epoch")
plt.legend()
plt.show()

plt.figure()
plt.plot(train_err_torch, label="PyTorch NN - Train MSE")
if any(e is not None for e in val_err_torch):
    plt.plot([e for e in val_err_torch if e is not None], label="PyTorch NN - Val MSE")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.title("PyTorch NeuralNet - Loss per epoch")
plt.legend()
plt.show()


NameError: name 'train_err_manual' is not defined

<Figure size 640x480 with 0 Axes>

In [ ]:

# Scatter plots: true vs predicted on TEST set


plt.figure(figsize=(15, 4))

# Manual BP
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_test_pred_manual, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         linestyle="--")
plt.xlabel("True cnt_log (test)")
plt.ylabel("Predicted")
plt.title("Manual BP")

# MLR
plt.subplot(1, 3, 2)
plt.scatter(y_test, y_test_pred_mlr, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         linestyle="--")
plt.xlabel("True cnt_log (test)")
plt.ylabel("Predicted")
plt.title("MLR (scikit-learn)")

# PyTorch NN
plt.subplot(1, 3, 3)
plt.scatter(y_test, y_test_pred_torch, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         linestyle="--")
plt.xlabel("True cnt_log (test)")
plt.ylabel("Predicted")
plt.title("PyTorch NN")

plt.tight_layout()
plt.show()


## Summary of model comparison

In this notebook we compared three regression models on the same
Bike Sharing hourly dataset:

- **Manual BP Neural Network** (custom implementation),
- **Multiple Linear Regression (MLR)** using scikit-learn,
- **PyTorch Neural Network** using the same architecture and hyperparameters as the manual BP model.

The main points are:

- All models were trained on the same **scaled** features and target.
- The manual BP and the PyTorch models used the same:
  - number of layers and neurons,
  - activation function,
  - learning rate, momentum,
  - number of epochs,
  - internal validation split.
- We measured performance using **MSE, MAE and MAPE** on:
  - the 80% train+validation set,
  - the 20% test set.

From the comparison tables and plots we can discuss:

- How the linear model (MLR) behaves compared to the nonlinear neural networks.
- Whether PyTorch reproduces or improves the performance of the manual BP implementation.
- How stable each model is during training (by looking at the loss curves).
- How close the predictions are to the true values on the test set (scatter plots).

